In [1]:
pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


In [2]:
pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (8,204 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [4]:
import pytesseract
import pandas as pd
import numpy as np
import re
import os
import easyocr
import cv2
import matplotlib.patches as patches
from PIL import Image

In [5]:
### this works Tesseract and easyOCR mix
#text extraction function
def extract_text_from_images(folder_path):
    # Defining path to tesseract.exe
    #path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    #path_to_tesseract = r"C:\Users\jbaggett\Anaconda3\envs\fastai3\Library\bin\tesseract.exe"
  
    # Providing the tesseract executable location to pytesseract library
    #pytesseract.tesseract_cmd = path_to_tesseract
    pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
    # Initialize an empty list to store the extracted text and filenames
    text_list = []
    files = os.listdir(folder_path)
    
    # Loop through all the files in the folder
    for file_name in files:
        # Check if the file is an image
        if file_name.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            # Construct the full path to the image file
            image_path = os.path.join(folder_path, file_name)
            
            # Open the image file and store it in an image object
            img = Image.open(image_path)
            
            # recast image as numpy array
            np_img = np.array(img)
            img = np.array(img)
            
            # if image has multiple channels convert it to grayscale (could probably improve this)
            if len(img.shape)>2:
                img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            
            # crop to include only bottom of image
            # note in most images the ultrasound part is between 10 and 665 for the first coord (y), and 2 and 853 for the second coord (x)
            img = img[ 500:, 22:833 ]
            
            # convert to black and white to increase contrast
            threshold = 160
            th,img = cv2.threshold(img,threshold,255,cv2.THRESH_BINARY)
            
            
            # show a dilated image
            #A larger kernel size will result in a more dilated image, which can make it easier to find contours.
            kernel = np.ones((7,7),np.uint8)
            img_dilated = cv2.dilate(img,kernel,iterations=5)
            contours,hierarchy = cv2.findContours(img_dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
            c = max(contours,key=cv2.contourArea)
            x,y,w,h = cv2.boundingRect(c)
            rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
            # fig,ax = plt.subplots()
            # ax.imshow(img_dilated)
            # ax.add_patch(rect)
            # fig.show()

            # Extract the text from the image using pytesseract
            text = pytesseract.image_to_string(img[y:y+h+1,x:x+w+1])
            text = text.replace("\n"," ")
            #print(len(text))
            
            # If the extracted text is empty, try extracting text from the entire image
            if not text or '¢' in text or len(text.split()) == 1:
                text = pytesseract.image_to_string(img)
                text = text.replace("\n"," ")
            if len(text) <= 5:
                reader = easyocr.Reader(['en'])
                result = reader.readtext(np_img, paragraph=True)
                #print(result)
                text = result[-1][-1]
                #print(text)
            # Append the extracted text and filename to the list
            text_list.append((file_name, text)) # exclude the last character which is usually a newline character
    
    # Create a pandas DataFrame from the text list
    df = pd.DataFrame(text_list, columns=['image_filename', 'text'])
    
    return df

In [6]:
folder_path = r"/content/drive/MyDrive/Colab Notebooks/image"
df = extract_text_from_images(folder_path)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/ocrPyTessract.csv', index=False)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [7]:
import pandas as pd
from google.colab import drive
import easyocr

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/textOnly.csv')

# Filter out rows with empty 'text' column
filtered_df = df[df['text'].isnull()]

# Iterate over the filtered DataFrame and retrieve image information
for index, row in filtered_df.iterrows():
    image_name = row['image_filename']
    image_path = '/content/drive/MyDrive/Colab Notebooks/image/' + image_name
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path, paragraph=True)
    filtered_df.at[index, 'text'] = result[-1][-1]

# Show the filtered DataFrame
filtered_df.shape


(24, 2)

In [ ]:
filtered_df.to_csv('/content/drive/MyDrive/Colab Notebooks/FilteredText.csv', index=False)

In [11]:
filtered_df

,image_filename,text
794,000795_cropped.png,TRANS RT BREAST 10:00 6CM FN
797,000798_cropped.png,TRANS RT BREAST 10:00 6CM FN
801,000802_cropped.png,LONG RT BREAST 10:00 6CM FN |
803,000804_cropped.png,LONG RT BREAST 10:00 6CM FN
881,000882_cropped.png,LONG RT BREAST 12:00 8CM FN
890,000891_cropped.png,11.30 12CM FN PALP
1456,001457_cropped.png,TRANS LT BREAST 10.00 1OCM FN
1614,001615_cropped.png,TRANS LT BREAST 10:00 4CM FN
1664,001665_cropped.png,LONG LT BREAST 1:30/7CM FN
1718,001719_cropped.png,TRANS RT BREAST 1:00 8CM FN
